# PyKX Query Components
Query all TorQ Components.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

from clusters import *

In [2]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# List of Clusters

In [3]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]
#    cdf = cdf[cdf['status'].isin(['RUNNING'])]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
1,discovery,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'discovery'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:10:34.715000+00:00,2024-03-01 20:54:59.373000+00:00,finspace-database,NaN
2,feed,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'tradeFeed'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:24:50.829000+00:00,2024-03-01 21:11:20.369000+00:00,finspace-database,NaN
3,gateway,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'gateway'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:24:49.462000+00:00,2024-03-01 21:11:17.301000+00:00,finspace-database,NaN
4,hdb,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'hdb'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '4'}]",Created with create_all notebook,2024-03-01 21:10:44.925000+00:00,2024-03-01 20:55:05.460000+00:00,finspace-database,NaN
5,rdb,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'rdb'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:10:44.563000+00:00,2024-03-01 20:55:02.410000+00:00,finspace-database,NaN


# Current State of Clusters

# Query the system through the Gateway

In [8]:
n='gateway'

conn = get_pykx_connection(client, 
                              environmentId=ENV_ID, clusterName=n, 
                              userName=KDB_USERNAME, boto_session=session)

queries = [
    '.gw.syncexec["meta trades";`rdb]',
    '.gw.syncexec["meta trades";`hdb]',
    '.gw.syncexec["select sum size by sym from trades";`rdb]',
    '.gw.syncexec["count trades";`rdb]',
    '.gw.syncexec["select [5] from trades";`rdb]',
    '.gw.syncexec["select [-5] from trades";`rdb]',
    '.gw.syncexec["select count i by date from trades";`hdb]', # FIXED
    '.gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]',
    '.gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]',
]

for q in queries:
    print(100*"=")
    print(f"Query: {q}")
    print(100*"-")
    display( conn(q).pd() )

Query: .gw.syncexec["meta trades";`rdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
sym,b's',,g
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["meta trades";`hdb]
----------------------------------------------------------------------------------------------------


,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
src,b's',,
price,b'f',,
size,b'i',,


Query: .gw.syncexec["select sum size by sym from trades";`rdb]
----------------------------------------------------------------------------------------------------


...

,x
date,
2014-04-21,29229
2014-04-22,26305
2014-04-23,32186


Query: .gw.syncexec["select sum size, max price by 0D00:05 xbar time from trades where sym=`IBM";`hdb`rdb]
----------------------------------------------------------------------------------------------------


,size,price
time,,
2014-04-21 08:00:00,106759,41.18
2014-04-21 08:05:00,105275,41.31
2014-04-21 08:10:00,53978,41.18
2014-04-21 08:15:00,76514,41.29
2014-04-21 08:20:00,114316,41.03
...,...,...
2014-04-23 16:20:00,97616,34.13
2014-04-23 16:25:00,138538,34.27
2024-03-01 21:20:00,81284,49.16


Query: .gw.syncexecj["select sum size by sym from trades";`rdb`hdb;sum]
----------------------------------------------------------------------------------------------------


,size
sym,
AAPL,25586000
CSCO,25799239
DELL,25609337
GOOG,26145638
IBM,25875980
MSFT,26257661
NOK,26054115
ORCL,25937062
YHOO,26203709


In [9]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-03-01 21:33:53.472937
